Import libraries and set up API key

In [25]:
import json
import requests
import pandas as pd
from datetime import datetime, timedelta

# Insert your API key here
API_KEY = '2fvIsbHL7chqQIfpyhaUzMwjy'
num_data_points = 100
start_date_str = '2023-01-01'

Get user input

In [26]:
# Insert your API key here
#API_KEY = str(input("Insert your API key here"))
#Ask user for input
#num_data_points = int(input("How many data points do you want to load? "))
#start_date = input("Enter the starting date (YYYY-MM-DD): ")

# Convert start date to Unix timestamp
start_datetime = datetime.strptime(start_date_str, "%Y-%m-%d")
start_timestamp = int(start_datetime.timestamp())

# Calculate end date and convert to Unix timestamp
end_datetime = start_datetime + timedelta(days=num_data_points)
end_timestamp = int(end_datetime.timestamp())

print(f"Start timestamp: {start_timestamp}")
print(f"End timestamp: {end_timestamp}")

Start timestamp: 1672543800
End timestamp: 1681180200


Define the function to fetch data

In [27]:
def fetch_glassnode_data(endpoint, asset='BTC'):
    url = f'https://api.glassnode.com/v1/metrics/{endpoint}'
    params = {
        'a': asset,
        'api_key': API_KEY,
        's': start_timestamp,
        'u': end_timestamp,
        'i': '24h'  # daily interval
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_json(response.text, convert_dates=['t'])
        column_name = endpoint.split('/')[-1]
        df = df.rename(columns={'t': 'timestamp', 'v': column_name})
        return df.set_index('timestamp')
    else:
        print(f"Error fetching data from {endpoint}: {response.status_code}")
        return pd.DataFrame()

Fetch price data

In [28]:
# Fetch price data
price_df = fetch_glassnode_data('market/price_usd_close')
price_df.head()

/var/folders/kw/s9r1_7lx3zn35g56dhqzfmwh0000gn/T/ipykernel_6628/1178596419.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(response.text, convert_dates=['t'])


,price_usd_close
timestamp,
2023-01-01,16620.819927
2023-01-02,16693.930009
2023-01-03,16682.715885
2023-01-04,16865.159755
2023-01-05,16841.821596


Define endpoints and fetch feature data

In [29]:
# List of endpoint URLs (add more as needed)
endpoints = [
    'indicators/sopr',
    'indicators/sopr_less_155',
    'indicators/sopr_account_based',
    'indicators/sopr_adjusted'
]

# Fetch and combine feature data
features_df = pd.DataFrame()
for endpoint in endpoints:
    df = fetch_glassnode_data(endpoint)
    if features_df.empty:
        features_df = df
    else:
        features_df = features_df.join(df, how='outer')

features_df.head()

/var/folders/kw/s9r1_7lx3zn35g56dhqzfmwh0000gn/T/ipykernel_6628/1178596419.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(response.text, convert_dates=['t'])
/var/folders/kw/s9r1_7lx3zn35g56dhqzfmwh0000gn/T/ipykernel_6628/1178596419.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(response.text, convert_dates=['t'])
/var/folders/kw/s9r1_7lx3zn35g56dhqzfmwh0000gn/T/ipykernel_6628/1178596419.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(response.text, convert_dates=['t'])
/var/folders/kw/s9r1_7lx3zn35g56dhqzfmwh0000gn/T/ipykernel_6628/1178596419.py:1

,sopr,sopr_less_155,sopr_account_based,sopr_adjusted
timestamp,,,,
2023-01-01,0.993143,0.995300,0.906646,0.982933
2023-01-02,0.997344,1.001048,0.913555,0.993102
2023-01-03,0.987792,0.995558,0.916211,0.971039
2023-01-04,0.985428,0.999432,0.892713,0.968075
2023-01-05,0.990950,0.997645,0.963089,0.978652


Combine data and handle missing values

In [30]:
# Concatenate price and features dataframes
combined_df = pd.concat([price_df, features_df], axis=1)

# Handle missing values
combined_df = combined_df.dropna()

# Display the result
combined_df.head()

,price_usd_close,sopr,sopr_less_155,sopr_account_based,sopr_adjusted
timestamp,,,,,
2023-01-01,16620.819927,0.993143,0.995300,0.906646,0.982933
2023-01-02,16693.930009,0.997344,1.001048,0.913555,0.993102
2023-01-03,16682.715885,0.987792,0.995558,0.916211,0.971039
2023-01-04,16865.159755,0.985428,0.999432,0.892713,0.968075
2023-01-05,16841.821596,0.990950,0.997645,0.963089,0.978652


 Additional analysis or visualization (optional)

In [31]:
# You can add more cells here for further analysis or visualization
combined_df.describe()

,price_usd_close,sopr,sopr_less_155,sopr_account_based,sopr_adjusted
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,23415.304798,1.000595,1.016027,1.009464,1.002454
std,3353.406517,0.016398,0.018120,0.052797,0.035659
min,16620.819927,0.880878,0.961018,0.883769,0.766452
25%,21804.378982,0.996025,1.003279,0.970866,0.990014
50%,23164.313442,1.003370,1.016252,1.018884,1.009144
75%,25526.203345,1.009071,1.027639,1.041921,1.019797
max,29628.544376,1.029593,1.076041,1.109142,1.070677
